#  causality sentence Identifier

## 1.  load sentence

In [2]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

pm_DB = pd.read_csv('data/in/KNN_RAG/semeval_causalityDetection_cleaned.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]

instances = pm_DB['sentence'].to_list()

len(instances)

549

## 2. load fewshot database

In [3]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()


In [4]:
semeval_data = semeval_data.dropna(subset=['causality_phrase'])
len(semeval_data)

#semeval_data = pd.read_csv('data/in/semeval_CE_pattern.csv')# data/in/ADE/ADE_CE_pattern.csv
causality_phrase = semeval_data['causality_phrase'].to_list()
len(causality_phrase)

2365

## 3. get causality label for sentence

In [5]:
from openai import OpenAI
client = OpenAI(api_key='enter your API key here..')


In [6]:
#pattern matcher function.
import random
from fuzzywuzzy import fuzz

def get_fewshot_examples_index(input_sentence) :

    #get few shot example index using Leiveinstein ratio.

    fewshot_index = []
    for ind in range (len(causality_phrase)):
        p = causality_phrase[ind]
        window_size = len(p)

        # Slide through input_sentence with a window size of 8
        for i in range(len(input_sentence) - window_size + 1):
            window = input_sentence[i:i + window_size]
            score = fuzz.ratio(window, p)
            if score >90:
                fewshot_index.append(ind)
                #print('p is ',p, 'i is', i)
                #print(f"Window '{window}' - Levenshtein Score: {score}")
    fewshot_index = list(set(fewshot_index))
    
    if len(fewshot_index)>10:
        fewshot_index = random.sample(fewshot_index, 10)
        #fewshot_index= fewshot_index[:10]
        
    
    return fewshot_index
    

#get all sentences and save as json example1

In [7]:

def get_fewshotexamples(fewshot_index):


    train_sentence = semeval_data['sentence'].to_list()

    fewshot_examples = [train_sentence[i] for i in fewshot_index]

    import json
    #fewshot_examples_json = {f'example {i + 1}': sentence for i, sentence in enumerate(fewshot_examples)}
    #fewshot_examples_json = json.dumps(fewshot_examples_json)
    #print(fewshot_examples)
    
    return fewshot_examples



In [8]:
#Below are 10 example sentences which shows causality relation  with  their cause and effect 
          #phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           #+ fewshot_examples
###model#gpt-3.5-turbo
def CS_Identifier(input_sentence, fewshot_examples):
    
    response = client.chat.completions.create(
      model="gpt-4o",
      response_format= {"type": "json_object"},
      messages=[
        {
          "role": "system",
          "content": '''
          You will be given a sentence, and your task is to identify whether the sentence shows a 
          causality relation or not. If the sentence shows a causality relation, you can output 1;
          if it doesn't, output 0. 
          Your output should be in json format: {output :  }.
          
          Hint: If the sentence shows causality relation, you will be able to identify a pair of 
          phrase in the sentence where one phrase will be the cause of the other phrase(effect).
          
          Below are around 20 example sentences which shows causality relation  with  their cause and effect 
          phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           '''+ fewshot_examples
          
            
        },
        {
          "role": "user",
          "content": input_sentence 
        }
      ],
      temperature=0.0,
      seed = 1,
      max_tokens=100,
      top_p=1
    )
    
    return response.choices[0].message.content


In [9]:
import json
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
causality_identifier=[]
fewshotfound_inputsentence_index=[]
fewshot_examples_list=[]
for i in range(len(pm_DB)):
    input_sentence = pm_DB['sentence'][i]
    
    #KNN+RAG few shot examples augmentation.
    fewshot_index = get_fewshot_examples_index(input_sentence)############FEWSHOT
    
    fewshot_examples_rag = get_fewshotexamples(fewshot_index) #############fewshot
    #fewshot_examples =  get_randomfewshotexamples()##################random shot/zero shot
        
    print('input_sentence : ', input_sentence)
    ###
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_examples_knn = [examples[j] for j in most_similar_indices]

   
    fewshot_comb = fewshot_examples_rag+fewshot_examples_knn
    fewshot_comb = {f'example {k + 1}': sentence for k, sentence in enumerate(fewshot_comb)}
    fewshot_comb = json.dumps(fewshot_comb)   

    print('fewshot_comb : ', fewshot_comb)
    ################# KNN+RAG end---
    
    #print('type', type(fewshot_examples_col))
    #print('')
    fewshot_examples_list.append(fewshot_comb)
    # output generation
    out = CS_Identifier(input_sentence, fewshot_comb)
    json_out = json.loads(out)
    causality_identifier_out = json_out['output']
    causality_identifier.append(int(causality_identifier_out))
    
    
    print(i)
    print(causality_identifier_out)
    
    

input_sentence :  Dogs develop a fever from stress and/or pain such as in a severe flea infestation.
fewshot_comb :  {"example 1": "\"Sip the tea slowly to reduce stomach <effect>pain</effect> from <cause>indigestion</cause>, bloating and feeling of fullness.\"", "example 2": "\"However, a <effect>headache</effect> from <cause>wine</cause> is often stronger and last longer due to several compounds.\"", "example 3": "\"The <effect>benefits</effect> from <cause>inflation</cause> which Professor Mulligan cites (e.g., bailing out underwater mortgagers) are better understood as a government sponsored transfer of wealth from savers to speculators.\"", "example 4": "\"He has a tattoo on his right arm and <effect>scars</effect> from <cause>stitches</cause> on his right elbow.\"", "example 5": "<effect>Sustained ventricular tachycardia</effect> in a <cause>thalidomide</cause>-treated patient with primary plasma-cell leukemia.", "example 6": "<effect>Leishmania infantum leishmaniasis</effect> in

2
1
input_sentence :  The fat and cholesterol cause heart disease; the animal protein causes cancer.
fewshot_comb :  {"example 1": "<cause>Quetiapine</cause> and <effect>obsessive-compulsive symptoms</effect> (OCS): case report and review of atypical antipsychotic-induced OCS.", "example 2": "<effect>Leishmania infantum leishmaniasis</effect> in <cause>corticosteroid</cause>--treated patients.", "example 3": "<cause>Asthma</cause> causes <effect>swelling</effect> and narrowing of the airways.", "example 4": "\"Furthermore, the substrate doping density at which impact <cause>ionization</cause> causes <effect>avalanche</effect> breakdown at the trench capacitor junction has been estimated.\"", "example 5": "The most commonly recognized toxic effect of <cause>ethambutol</cause> is <effect>optic neuropathy</effect>, which generally is considered uncommon and reversible in medical literature.", "example 6": "\"Snails and <cause>slugs</cause> cause <effect>damage</effect> to seedlings, flowe

6
0
input_sentence :  Social breakdown and poverty caused by civil war in several African countries has caused further increases in the rate of prostitution in those countries.
fewshot_comb :  {"example 1": "\"American skiier Lindsey Vonn denied downhill treble as <cause>fog</cause> caused <effect>cancellation</effect> in Val d'Isere.\"", "example 2": "Severe <cause>dapsone</cause> <effect>hypersensitivity syndrome</effect>.", "example 3": "A <effect>paradoxical ocular effect</effect> of <cause>brimonidine</cause>.", "example 4": "OBJECTIVE: We report a patient who developed neutropenia on <cause>clozapine</cause>, but behind the cell count decrease showed to be a <effect>diurnal variation of the white blood cells</effect> (WBC).", "example 5": "\"The weekend <cause>snowstorm</cause> caused <effect>delays</effect> that had some passengers still stuck in Washington, DC on Monday night.\"", "example 6": "\"Their chemical <effect>activation</effect> by <cause>light</cause> (halogen, plasm

9
0
input_sentence :  Resulting congestion caused delays of 15-20 minutes to Northeast Corridor, North Jersey Coast Line and Midtown Direct trains.
fewshot_comb :  {"example 1": "Severe <cause>dapsone</cause> <effect>hypersensitivity syndrome</effect>.", "example 2": "BACKGROUND: reports on <effect>delated cutaneous reactions</effect> to <cause>captopril</cause> have been seldom reported.", "example 3": "\"The <cause>compromises</cause> contained in the agreement caused the civil <effect>war</effect> in the 26 counties in June 1922-April 1923, in which the pro-Treaty Provisional Government defeated the anti-Treaty Republican forces.\"", "example 4": "\"Frustrations, <cause>threats</cause>, and conflicts cause <effect>stress</effect>.\"", "example 5": "\"Both <cause>colds</cause> and flu cause <effect>inflammation</effect> of the mucous membranes (found in the nose, throat and mouth).\"", "example 6": "<effect>Fixed drug eruption</effect> to <cause>rofecoxib</cause>.", "example 7": "\"S

12
1
input_sentence :  Already by the middle of the eighteenth century, several small clockmaking shops produced cuckoo clocks with wooden gears.
fewshot_comb :  {"example 1": "It was also felt necessary at this time to draw up a similar document, dealing not with the prevention of <effect>marine pollution</effect> by <cause>dumping</cause>, but instead with the prevention of marine pollution by discharges of dangerous substances from land-based sources, watercourses or pipelines.", "example 2": "\"On both of my systems, the <cause>reboot</cause> produced the ominous <effect>message</effect> 'Missing operating system'.\"", "example 3": "Although adverse effects are usually mild, the author reports here a case of <effect>leucocytopenia</effect> and thrombocytopenia with <cause>quetiapine</cause> treatment that required its discontinuation.", "example 4": "<cause>The front wheels</cause> are making <effect>a grinding noise</effect>.", "example 5": "\"Their chemical <effect>activation</ef

...

536
1
input_sentence :  The European colonial expansion to the Americas resulted in massive population movements from the Old World to the New World.
fewshot_comb :  {"example 1": "We present an AIDS patient with severe and prolonged <effect>lactic acidosis</effect> on stavudine and <cause>lamivudine</cause>.", "example 2": "\"The <cause>violence</cause> resulted in the <effect>death</effect> of police \"capo\" Filippo Raciti, which caused widespread soul-searching and a week-long suspension of Italian football.\"", "example 3": "\"An alert victim with a broken neck or severely torn ligament has enough <effect>discomfort</effect> from the <cause>injury</cause> and muscle spasm to force him to hold his neck still.\"", "example 4": "<effect>Lichenoid drug eruption</effect> to <cause>salsalate</cause>.", "example 5": "<effect>Transient asymptomatic bradycardia</effect> in patients on infusional <cause>5-fluorouracil</cause>.", "example 6": "<cause>The dispute</cause> has resulted in <effe

539
0
input_sentence :  The song was composed for a famous Brazilian musician.
fewshot_comb :  {"example 1": "We present an AIDS patient with severe and prolonged <effect>lactic acidosis</effect> on stavudine and <cause>lamivudine</cause>.", "example 2": "Physicians should be cognizant of this potential complication in patients receiving <cause>thalidomide</cause> or <cause>thalidomide</cause>-like drugs who present with <effect>fever</effect> and pulmonary infiltrates and fail to improve despite treatment with broad-spectrum antibiotics.", "example 3": "<effect>Scleromyxedema</effect> in a patient with multiple sclerosis and monoclonal gammopathy on <cause>interferon beta-1a</cause>.", "example 4": "OBJECTIVE: We report a patient who developed neutropenia on <cause>clozapine</cause>, but behind the cell count decrease showed to be a <effect>diurnal variation of the white blood cells</effect> (WBC).", "example 5": "This is the second report of <effect>lactic acidosis</effect> in a pati

542
0
input_sentence :  In another two short sessions, the duo produced six or seven songs, two of which just didn't fit the mood of the album.
fewshot_comb :  {"example 1": "This is the second report of <effect>lactic acidosis</effect> in a patient on stavudine and <cause>lamivudine</cause>.", "example 2": "A <effect>paradoxical ocular effect</effect> of <cause>brimonidine</cause>.", "example 3": "\"The enclosed <cause>community</cause> produced the enclosed <effect>personality</effect>, and vice versa.\"", "example 4": "<effect>Neurotoxicity</effect> of <cause>valacyclovir</cause> in peritoneal dialysis: a pharmacokinetic study.", "example 5": "\"The <cause>bombing</cause> produced the worst <effect>carnage</effect> since a Nov. 23 attack in the Shi'ite slum of Sadr City that killed more than 200 people.\"", "example 6": "<effect>Neurotoxicity</effect> of intrathecal <cause>methotrexate</cause>: MR imaging findings.", "example 7": "Physicians should be cognizant of this potential com

545
1
input_sentence :  The movie featured a mutated sheep as the titular monster, which grew larger throughout the movie, until it exploded in the climactic scene.
fewshot_comb :  {"example 1": "<effect>Scleromyxedema</effect> in a patient with multiple sclerosis and monoclonal gammopathy on <cause>interferon beta-1a</cause>.", "example 2": "<effect>Energy expenditure</effect> through <cause>physical activity</cause> is a key component in the prevention and treatment of overweight and obesity.", "example 3": "\"We examined mood <effect>alterations</effect> through physical <cause>activity</cause> and other interventions.\"", "example 4": "A case of <effect>heatstroke</effect> is reported in a 32-year-old man diagnosed with schizophrenia and on <cause>clozapine</cause> monotherapy.", "example 5": "\"<effect>Inhibition</effect> through synaptic <cause>depression</cause> is unlike the previous forms of inhibition in that it turns on more slowly and thus acts as delayed negative feedback.

548
1


In [10]:
1524

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


1524

In [11]:
len(causality_identifier)

549

In [12]:
pm_DB['GPT_causality_label'] = causality_identifier
#pm_DB['fewshot_examples'] = fewshot_examples_list

#pm_DB['patternmatcher_label'] = pattern_matched 
pm_DB

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label
0,0,0,0,Dogs develop a fever from stress and/or pain s...,1,"[-0.0036133944522589445, 0.0012565960641950369...",1
1,1,1,1,The new pencil and graphite drawing is the pai...,0,"[-0.02040475234389305, 0.022646330296993256, 0...",0
2,2,2,2,The deleveraging triggered by the crisis impli...,1,"[-0.02002304047346115, -0.031225474551320076, ...",1
3,3,3,3,The fat and cholesterol cause heart disease; t...,1,"[0.002320877043530345, -0.014957467094063759, ...",1
4,4,4,4,"The researchers placed the compound in a tube,...",0,"[-0.019478240981698036, 0.0026719982270151377,...",0
...,...,...,...,...,...,...,...
544,544,621,621,Neuralgia (TN) is the most common cause of chr...,1,"[-0.010079591535031796, 0.014008276164531708, ...",1
545,545,622,622,Muscle fatigue is the number one cause of arm ...,1,"[-0.011783338151872158, -0.007658560294657946,...",1
546,546,623,623,The movie featured a mutated sheep as the titu...,0,"[-0.015397767536342144, -0.01711292937397957, ...",0
547,547,624,624,A plasma was generated by an electric discharg...,1,"[-0.011884856037795544, 0.02052135579288006, 0...",1


In [13]:
#get accuracy for fewshot_semeval_causalityDetection: b/w label and GPT_causality_label 

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

# Calculate precision, recall, and F1 score
precision = precision_score(pm_DB['label'], pm_DB['GPT_causality_label'])
recall = recall_score(pm_DB['label'], pm_DB['GPT_causality_label'])
f1 = f1_score(pm_DB['label'], pm_DB['GPT_causality_label'])
accuracy = accuracy_score(pm_DB['label'], pm_DB['GPT_causality_label'])

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.9089253187613844
F1 Score: 0.8966942148760331
Precision: 0.9234042553191489
Recall: 0.8714859437751004


In [14]:

#filtered_rows = pm_DB[(pm_DB['GPT_causality_label'] == 1) & (pm_DB['patternmatcher_label'] == 1)]
#filtered_rows.to_csv('data/out/alzeimers_causalityrelation.csv')

#pm_DB.to_csv('data/out/RAG_causality/semeval/fewshotcomb_semeval_causalityDetection.csv')

In [ ]:
#pm_DB.to_csv('data/out/RAG_causality/task1/GPT4/zero_SCITE_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem

In [ ]:
pm_DB.to_csv('data/out/RAG_causality/task1/errorAnalysis/RAG_sem_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem